In [2]:
import numpy as np
import os
import matplotlib.pyplot as plt


In [3]:
cd desktop


C:\Users\Anil\desktop


In [12]:
dir = r"C:\Users\Anil\Desktop\leapGestRecog"

In [13]:
dir

'C:\\Users\\Anil\\Desktop\\leapGestRecog'

In [17]:
for folder in os.listdir(dir):
    for subfolder in os.listdir(dir + "\\" + folder):
        if "01_palm" in subfolder or "03_fist"in subfolder:
            print(subfolder)


01_palm
03_fist
01_palm
03_fist
01_palm
03_fist
01_palm
03_fist
01_palm
03_fist
01_palm
03_fist
01_palm
03_fist
01_palm
03_fist
01_palm
03_fist
01_palm
03_fist
